# Lesson 1: Getting Started with PaLM (using chat)

#### Setup

To use the PaLM API on your own machine, first install the library:
```Python
!pip install -q google.generativeai
```
The optional flag `-q` installs "quietly" without printing out details of the installation.

Note that this a a copy of the original Lesson 1 notebook where I changed the model to a chat model instead of the original text model.


In [3]:
from dotenv import load_dotenv
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib

load_dotenv()

palm.configure(
    api_key=os.environ.get("PaLM_API_KEY"),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

### Explore the available models

In [4]:
for m in palm.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

name: models/chat-bison-001
description: Chat-optimized generative language model.
generation methods:['generateMessage', 'countMessageTokens']

name: models/text-bison-001
description: Model targeted for text generation.
generation methods:['generateText', 'countTextTokens', 'createTunedTextModel']

name: models/embedding-gecko-001
description: Obtain a distributed representation of a text.
generation methods:['embedText']



#### Filter models by their supported generation methods
- `generateText` is currently recommended for coding-related prompts.
- `generateMessage` is optimized for multi-turn chats (dialogues) with an LLM.

In [5]:
models = [m for m in palm.list_models() 
          if 'generateMessage' 
          in m.supported_generation_methods]
models

[Model(name='models/chat-bison-001', base_model_id='', version='001', display_name='Chat Bison', description='Chat-optimized generative language model.', input_token_limit=4096, output_token_limit=1024, supported_generation_methods=['generateMessage', 'countMessageTokens'], temperature=0.25, top_p=0.95, top_k=40)]

In [6]:
model_bison = models[0]
model_bison

Model(name='models/chat-bison-001', base_model_id='', version='001', display_name='Chat Bison', description='Chat-optimized generative language model.', input_token_limit=4096, output_token_limit=1024, supported_generation_methods=['generateMessage', 'countMessageTokens'], temperature=0.25, top_p=0.95, top_k=40)

#### helper function to generate text

- The `@retry` decorator helps you to retry the API call if it fails.
- We set the temperature to 0.0 so that the model returns the same output (completion) if given the same input (the prompt).

In [7]:

from google.api_core import retry
@retry.Retry()
def generate_message(prompt,
                  model=model_bison,
                  temperature=0.0):

     return palm.chat(messages=prompt, temperature=temperature
#    return palm.generate_text(prompt=prompt, temperature=temperature,
#
# these only work with the generate_text method                              
#              safety_settings=[
#        {
#            "category": palm.types.HarmCategory.HARM_CATEGORY_DEROGATORY,
#            "threshold": palm.types.HarmBlockThreshold.BLOCK_NONE
#        }
#    ]      
                    
    )

#### Ask the LLM how to write some code



In [19]:
prompt = "show me  code to print out a linked list in Python"
completion = generate_message(prompt)
if (completion.last == None):
    print(completion)
else:
    print(completion.last)
#print(completion.result)

Sure, here is a simple code to print out a linked list in Python:

```python
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, data):
        new_node = Node(data)
        if self.head is None:
            self.head = new_node
        else:
            node = self.head
            while node.next is not None:
                node = node.next
            node.next = new_node

    def print_list(self):
        node = self.head
        while node is not None:
            print(node.data)
            node = node.next

if __name__ == "__main__":
    list = LinkedList()
    list.append(1)
    list.append(2)
    list.append(3)
    list.print_list()
```

This code will print out the following:

```
1
2
3
```


- Tip: The words "show me" tends to encourage the PaLM LLM to give more details and explanations compared to if you were to ask "write code to ..."

In [24]:
prompt = "write code to iterate across a linked list in Python"
completion = generate_message(prompt)
if (completion.last == None):
    print("Failed\n")
    print(completion)
else:
    print(completion.last)

Failed

ChatResponse(model='models/chat-bison-001', context='', examples=[], messages=[{'author': '0', 'content': 'write code to iterate across a linked list in Python'}, None], temperature=0.0, candidate_count=None, candidates=[], filters=[{'reason': <BlockedReason.OTHER: 2>}], top_p=None, top_k=None)


#### Try out the code
- Try copy-pasting some of the generated code and running it in the notebook.
- Remember to test out the LLM-generated code and debug it make sure it works as intended.

In [12]:
# paste the LLM's code here# Create a list
list = ["apple", "banana", "cherry"]

# Iterate over the list using a for loop
for item in list:
    print(item)




apple
banana
cherry


#### Try asking your own coding question

In [ ]:
# Modify the prompt with your own question
prompt = "Show me how to [...]"

completion = generate_text(prompt)

#### Note about the API key
We've provided an API key for this classroom.  If you would like your own API key for your own projects, you can get one at [developers.generativeai.google](https://developers.generativeai.google/)